## merge model

In [1]:
## 合并模型
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM, TextIteratorStreamer, GenerationConfig,LlamaTokenizer
import torch
from peft import PeftModel
import os
lora_path='/workspace/ye/DPO/checkpoint/llama2/goverment_6000step_que2res'
output_path='/workspace/ye/DPO/checkpoint/llama2/goverment_6000step_que2res/merged_model'
base_model='/workspace/ye/llm/llama2-7b'
base = AutoModelForCausalLM.from_pretrained(
        base_model, torch_dtype=torch.float16, low_cpu_mem_usage=True
    )
base_tokenizer = LlamaTokenizer.from_pretrained(base_model)

print(f"Loading the LoRA adapter from {lora_path}")

lora_model = PeftModel.from_pretrained(
    base,
    lora_path,
    torch_dtype=torch.float16,
)

print("Applying the LoRA")
model = lora_model.merge_and_unload()
output_merged_dir = os.path.join(output_path, "pytorch_model.bin")
torch.save(model.state_dict(),output_merged_dir)
print(f"Saving the target model to {output_path}")
# model.save_pretrained(output_path)
# base_tokenizer.save_pretrained(output_path)

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 11.04it/s]


Loading the LoRA adapter from /workspace/ye/DPO/checkpoint/llama2/goverment_6000step_que2res
Applying the LoRA
Saving the target model to /workspace/ye/DPO/checkpoint/llama2/goverment_6000step_que2res/merged_model


## Test model

In [ ]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForCausalLM, TextIteratorStreamer, GenerationConfig,LlamaTokenizer,LlamaForCausalLM
model_base = LlamaForCausalLM.from_pretrained('/workspace/ye/DPO/checkpoint/llama2/gsm8k')
base_tokenizer = LlamaTokenizer.from_pretrained('/workspace/ye/DPO/checkpoint/llama2/dpo-hh-5e5-beta-0.5-ep1/checkpoint-500')

In [ ]:
from datasets import  load_dataset
# data = load_dataset('/workspace/ye/DPO/Gsm8k/finetune')
data = load_dataset('/workspace/ye/DPO/Gsm8k/evaluation')
model_base.half()
model_base.to('cuda:6')

In [ ]:
result =[]
from tqdm import tqdm
for i in tqdm(range(10)):
    inputs = base_tokenizer.batch_encode_plus(["Question:"+data['train']['question'][i]+'\n\nAnswer:'], return_tensors="pt",padding=True).to('cuda:6')
    outputs = model_base.generate(**inputs,max_new_tokens=512,do_sample=False,eos_token_id=2)
    # print(len(outputs[0].tolist()))
    # print(outputs[0].tolist())
    response = base_tokenizer.decode(outputs[0].tolist()[len(inputs['input_ids'][0]):])
    result.append([response])
    print(response)
c=0
for i in range(len(result)):
    if result[i][0].strip('</s>').split('#### ')[-1] == result[i][1].strip('</s>').split('#### ')[-1]:
        c+=1
c/len(result)

In [1]:
from transformers import  AutoModel,AutoTokenizer,LlamaTokenizer,LlamaForCausalLM
import os
output_dir = '/workspace/ye/DPO/checkpoint/llama2/dpo-hh-5e5-beta-0.5-ep1/checkpoint-1000'
from datasets import  load_dataset
data = load_dataset('/workspace/ye/DPO/Gsm8k/finetune')
# data = load_dataset('/workspace/ye/dataset/processed_Dahoas/full_hh_rlhf')
class Simple_Generate():
    def __init__(self,model_path):
        self.model= LlamaForCausalLM.from_pretrained(model_path,trust_remote_code=True).half().to('cuda:7')
        self.tokenizer = LlamaTokenizer.from_pretrained(model_path, trust_remote_code=True)
    def forward(self,prompt):
        inputs = self.tokenizer.batch_encode_plus(["Question:"+prompt+'\n\nAnswer:'], return_tensors="pt",padding=True).to('cuda:7')
        # print(inputs)
        outputs = self.model.generate(**inputs,max_new_tokens=1024,do_sample=True,eos_token_id=2)
        print(len(outputs[0].tolist()))
        print(outputs[0].tolist())
        response = self.tokenizer.decode(outputs[0].tolist()[len(inputs['input_ids'][0]):])
        return response
Simple_Generate = Simple_Generate(os.path.join(output_dir))

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 7473 examples [00:00, 255677.64 examples/s]
Generating test split: 1319 examples [00:00, 576159.86 examples/s]
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Loading checkpoint shards: 100%|██████████| 3/3 [00:15<00:00,  5.21s/it]


In [2]:
data = load_dataset('/workspace/ye/DPO/Gsm8k/evaluation')

Generating test split: 1319 examples [00:00, 345746.33 examples/s]


In [4]:
data

DatasetDict({
    test: Dataset({
        features: ['question', 'response_j', 'response_k'],
        num_rows: 1319
    })
})

In [ ]:
Simple_Generate.forward("James decides to run 3 sprints 3 times a week.  He runs 60 meters each sprint.  How many total meters does he run a week?")

In [5]:
for i in range(20,30):
    print(Simple_Generate.forward(data['test']['question'][i]))

277
[1, 894, 29901, 29902, 505, 29871, 29896, 29900, 11872, 414, 310, 24841, 13748, 393, 526, 1023, 29899, 22585, 29879, 4094, 322, 306, 6398, 304, 788, 372, 304, 29871, 29896, 29945, 11872, 414, 310, 282, 457, 11548, 13748, 393, 338, 2211, 29899, 28491, 386, 29879, 4094, 29889, 1205, 408, 306, 1671, 372, 29892, 306, 805, 453, 697, 4631, 310, 278, 24841, 13748, 29889, 1128, 1568, 4094, 338, 297, 278, 9886, 29871, 29906, 29946, 11872, 414, 29973, 13, 13, 22550, 29901, 450, 9886, 29871, 29906, 29946, 11872, 414, 3743, 29871, 29896, 29945, 11872, 414, 310, 282, 457, 11548, 13748, 322, 29871, 29929, 11872, 414, 310, 24841, 13748, 29889, 13, 13, 7900, 22137, 1346, 22550, 29901, 29871, 7027, 29991, 1334, 508, 671, 445, 304, 8147, 777, 901, 2712, 29889, 960, 591, 22932, 278, 29871, 29929, 11872, 414, 310, 24841, 13748, 491, 278, 8666, 310, 24841, 13748, 29892, 591, 679, 278, 8666, 310, 29871, 29929, 11872, 414, 310, 24841, 13748, 29889, 1987, 565, 591, 22932, 278, 29871, 29896, 29945, 11872, 

## Dataprocess

In [7]:
import json
import pandas as pd
with open('/workspace/ye/SFT/infer_result/204/SFT-llama2-test-dpo-set_greedy/Gsm8K/0.00%Acc.json',"r",encoding='utf-8') as f:
    data = json.load(f)
question = []
answer = []
reject = []
for i in data:
    question.append(i['question'])
    answer.append(i['answer'])
    reject.append(i['response'][0].strip('</s>'))
df = pd.DataFrame({'question':question, 'response_j': answer,'response_k':reject})
df.to_parquet('/workspace/ye/DPO/Gsm8k/iter1/Gsm8K-test-set.parquet')


In [ ]:
data
c = 0
for i in data:
    if i['answer'].split('#### ')[-1] == i['response'][0].split('#### ')[-1].strip('</s>'):
        c+=1
    else:
        # print(i['answer'].split('#### ')[-1] ,i['response'][0].split('#### ')[-1].strip('</s>'))
        pass
    
    # print(i['answer'].split('#### ')[-1],i['response'][0].split('#### ')[-1].strip('</s>'))
c/len(data)

In [ ]:
from datasets import load_dataset
dataset=load_dataset('/workspace/ye/dataset/Dahoas/full_hh_rlhf',split='train',data_dir=None)
original_columns = dataset.column_names
num_proc=24
def return_prompt_and_responses(samples):
    return {
        "prompt": ["Question: " + question + "\n\nAnswer: " for question in samples["prompt"]],
        "chosen":  [sample +'<endoftext>'for sample in samples["chosen"]],
        "rejected":[sample +'<endoftext>'for sample in samples["rejected"]]
    }
data=dataset.map(
        return_prompt_and_responses,
        batched=True,
        num_proc=24,
        remove_columns=original_columns,
    )
eval_dataset = data.filter(
    lambda x: len(x["prompt"]) + len(x["chosen"]) <= 1024
    and len(x["prompt"]) + len(x["rejected"]) <= 1024
)

In [ ]:
from transformers import LlamaForCausalLM,LlamaTokenizer
import math
tokenizer = LlamaTokenizer.from_pretrained('/workspace/ye/llm/llama2-7b')
tokenizer.add_special_tokens(
            {'additional_special_tokens': ['<endoftext>']})
tokenizer('<endoftext>')
model.resize_token_embeddings(int(
    8 *
    math.ceil(len(tokenizer) / 8.0)))  # make the vocab size multiple of 8

In [ ]:
/workspace/ye/SFT/infer_result/203/SFT-llama2-train-dpo-set_greedy/Gsm8K/0.00%Acc.json

In [ ]:
/workspace/ye/SFT/infer_result/204/SFT-llama2-test-dpo-set_greedy/Gsm8K/0.00%Acc.json